<a href="https://colab.research.google.com/github/SureshTakasi/DATA602-ML-UMBC/blob/main/Amazon_Product_Review_Sentiment_Analysis_and_Star_Rating_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import sys
import os
import gzip

"""
We will ignore FutureWarning and DeprecationWarning
"""
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

"""
We will ignore warnings
"""
warnings.filterwarnings("ignore")

if not sys.warnoptions:
    warnings.simplefilter("ignore")
program_start_time=time.time()

Download dataset

In [15]:
!wget https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz

--2020-10-08 14:53:27--  https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.184.93
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.184.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22870508 (22M) [application/x-gzip]
Saving to: ‘amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz.1’

amazon_reviews_us_M 100%[===================>]  21.81M  33.0MB/s    in 0.7s    

2020-10-08 14:53:28 (33.0 MB/s) - ‘amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz.1’ saved [22870508/22870508]



Open dataset and save as dataframe

In [16]:
import gzip
with gzip.open('amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz') as f:
    data = pd.read_csv(f, sep='\t', error_bad_lines=False)

data.head()

b'Skipping line 35246: expected 15 fields, saw 22\n'
b'Skipping line 87073: expected 15 fields, saw 22\n'


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,20422322,R8MEA6IGAHO0B,B00MC4CED8,217304173,BlackVue DR600GW-PMP,Mobile_Electronics,5.0,0.0,0.0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31
1,US,40835037,R31LOQ8JGLPRLK,B00OQMFG1Q,137313254,GENSSI GSM / GPS Two Way Smart Phone Car Alarm...,Mobile_Electronics,5.0,0.0,1.0,N,Y,five star,it's great,2015-08-31
2,US,51469641,R2Y0MM9YE6OP3P,B00QERR5CY,82850235,iXCC Multi pack Lightning cable,Mobile_Electronics,5.0,0.0,0.0,N,Y,great cables,These work great and fit my life proof case fo...,2015-08-31
3,US,4332923,RRB9C05HDOD4O,B00QUFTPV4,221169481,abcGoodefg® FBI Covert Acoustic Tube Earpiece ...,Mobile_Electronics,4.0,0.0,0.0,N,Y,Work very well but couldn't get used to not he...,Work very well but couldn't get used to not he...,2015-08-31
4,US,44855305,R26I2RI1GFV8QG,B0067XVNTG,563475445,Generic Car Dashboard Video Camera Vehicle Vid...,Mobile_Electronics,2.0,0.0,0.0,N,Y,Cameras has battery issues,"Be careful with these products, I have bought ...",2015-08-31


#### Data Preprocessing

Check info of dataset

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104852 entries, 0 to 104851
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   marketplace        104852 non-null  object 
 1   customer_id        104852 non-null  int64  
 2   review_id          104852 non-null  object 
 3   product_id         104852 non-null  object 
 4   product_parent     104852 non-null  int64  
 5   product_title      104852 non-null  object 
 6   product_category   104852 non-null  object 
 7   star_rating        104850 non-null  float64
 8   helpful_votes      104850 non-null  float64
 9   total_votes        104850 non-null  float64
 10  vine               104850 non-null  object 
 11  verified_purchase  104850 non-null  object 
 12  review_headline    104848 non-null  object 
 13  review_body        104849 non-null  object 
 14  review_date        104850 non-null  object 
dtypes: float64(3), int64(2), object(10)
memory usage: 1

Check shape of data

In [18]:
data.shape

(104852, 15)

Describe the data

In [19]:
data.describe()

,customer_id,product_parent,star_rating,helpful_votes,total_votes
count,1.048520e+05,1.048520e+05,104850.000000,104850.000000,104850.000000
mean,2.793783e+07,5.015196e+08,3.763176,1.243996,1.615393
std,1.508722e+07,2.871661e+08,1.523526,7.070387,7.909896
min,1.007100e+04,5.352400e+04,1.000000,0.000000,0.000000
25%,1.471402e+07,2.593731e+08,3.000000,0.000000,0.000000
50%,2.650357e+07,4.937289e+08,4.000000,0.000000,0.000000
75%,4.223551e+07,7.440083e+08,5.000000,1.000000,1.000000
max,5.309657e+07,9.999508e+08,5.000000,769.000000,791.000000


Convert type of columns star_rating, helpful_votes and total_votes from float to int

In [20]:
cols = ['star_rating','helpful_votes', 'total_votes']
data[cols] = data[cols].astype('Int64')
data['star_rating'].head()

0    5
1    5
2    5
3    4
4    2
Name: star_rating, dtype: Int64

Check null values

In [21]:
data.isnull().sum()

marketplace          0
customer_id          0
review_id            0
product_id           0
product_parent       0
product_title        0
product_category     0
star_rating          2
helpful_votes        2
total_votes          2
vine                 2
verified_purchase    2
review_headline      4
review_body          3
review_date          2
dtype: int64

Check whether review_id is unique

In [27]:
data.review_id.value_counts().sum()

104852

The count of unique review_id equals the number of rows. Hence review_id is unique.

Create a new dataframe with review_id as index.
Remove two columns 'marketplace' and 'product_category' which is common for all the records. 

In [30]:
review_df = data.drop(columns=['marketplace', 'product_category'])
review_df.set_index('review_id', inplace=True)
review_df.head()

,customer_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
review_id,,,,,,,,,,,,
R8MEA6IGAHO0B,20422322,B00MC4CED8,217304173,BlackVue DR600GW-PMP,5,0,0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31
R31LOQ8JGLPRLK,40835037,B00OQMFG1Q,137313254,GENSSI GSM / GPS Two Way Smart Phone Car Alarm...,5,0,1,N,Y,five star,it's great,2015-08-31
R2Y0MM9YE6OP3P,51469641,B00QERR5CY,82850235,iXCC Multi pack Lightning cable,5,0,0,N,Y,great cables,These work great and fit my life proof case fo...,2015-08-31
RRB9C05HDOD4O,4332923,B00QUFTPV4,221169481,abcGoodefg® FBI Covert Acoustic Tube Earpiece ...,4,0,0,N,Y,Work very well but couldn't get used to not he...,Work very well but couldn't get used to not he...,2015-08-31
R26I2RI1GFV8QG,44855305,B0067XVNTG,563475445,Generic Car Dashboard Video Camera Vehicle Vid...,2,0,0,N,Y,Cameras has battery issues,"Be careful with these products, I have bought ...",2015-08-31


Check where the values are null

In [31]:
null_data = review_df[review_df.isnull().any(axis=1)]
null_data

,customer_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
review_id,,,,,,,,,,,,
R1M2HPGTLTOBZF,16892694,B000OFNQXE,139730060,Performance Teknique -- Performance Teknique I...,5,0,0,N,N,Five Stars,NaN,2014-11-11
R31BF4G9M6MJYE,33678324,B0091VOOPQ,196563098,Ultimate Arms Gear Tactical Combo Combination ...,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN
R2RCYG6GRJ75UT,47816860,B0091VOOPQ,196563098,Ultimate Arms Gear Tactical Combo Combination ...,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN
R392344S6XTCNF,48437078,B0052RMI2Y,423996186,USB Power Wall Charger + Syn Data Cable for Ap...,5,0,0,N,Y,NaN,met standards of criteria. #1 product for the ...,2012-11-24
R2YV0N3E97QR8H,48109029,B005HBSF68,455218203,Brand DW-D1000B Car 10-Inch Hd LCD TFT Stand A...,3,0,0,N,Y,NaN,You get what you pay for. Construction quality...,2012-09-05


We can see the that the rows with review ids R31BF4G9M6MJYE and R2RCYG6GRJ75UT have null values which are important for future predictions. Hence we can remove the two records.

In [38]:
review_df.dropna(subset=['review_headline', 'review_body'], how='all', inplace=True)

In [39]:
review_df.shape

(104850, 12)

Two rows have been removed where the both review headline and review body are not present.